#### Matching part

Balance Constrains: Balancing univariate moments by introducing additional decision 

In [7]:
import gurobipy as gp
from gurobipy import GRB

# Define sets
T = [1, 2, 3]  # Treated units
C = [4, 5, 6, 7, 8, 9]  # Control units
I = [1, 2]  # Covariates
m = 2  # Number of controls per treated unit

# Example distances δ_t,c
delta = {(t, c): abs(t - c) for t in T for c in C}

# Example covariate values (randomized for illustration)
x = {
    (4, 1): 10, (4, 2): 20, (5, 1): 15, (5, 2): 25,
    (6, 1): 12, (6, 2): 22, (7, 1): 11, (7, 2): 21,
    (8, 1): 14, (8, 2): 24, (9, 1): 13, (9, 2): 23
}

# Mean covariate values for treated group (example values)
x_T = {1: 13, 2: 23}  # Mean for each covariate in treated group

# Covariate importance weights (example)
omega = {1: 1.0, 2: 1.0}

# Create Gurobi model
model = gp.Model("Optimal_Matching_Balance")

# Decision variables
a = model.addVars(T, C, vtype=GRB.BINARY, name="a")
z = model.addVars(I, vtype=GRB.CONTINUOUS, name="z")

# Objective function: minimize sum of distances and covariate imbalance
model.setObjective(
    gp.quicksum(delta[t, c] * a[t, c] for t in T for c in C) +
    gp.quicksum(omega[i] * z[i] for i in I),
    GRB.MINIMIZE
)

# Constraint: Each treated unit is matched to exactly m controls
for t in T:
    model.addConstr(gp.quicksum(a[t, c] for c in C) == m, f"Match_{t}")

# Constraint: Each control is used at most once
for c in C:
    model.addConstr(gp.quicksum(a[t, c] for t in T) <= 1, f"Control_{c}")

# Constraint: Covariate balance constraints
for i in I:
    model.addConstr(
        z[i] >= gp.quicksum(a[t, c] * x[c, i] for t in T for c in C) / (m * len(T)) - x_T[i],
        f"Balance_Pos_{i}"
    )
    model.addConstr(
        z[i] >= -gp.quicksum(a[t, c] * x[c, i] for t in T for c in C) / (m * len(T)) + x_T[i],
        f"Balance_Neg_{i}"
    )

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal Matching Found:")
    for t in T:
        for c in C:
            if a[t, c].x > 0.5:  # Selected Matches
                print(f"Treated {t} matched with Control {c}")
    print("\nCovariate imbalance values:")
    for i in I:
        print(f"z[{i}] = {z[i].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 20 columns and 112 nonzeros
Model fingerprint: 0x0a26da86
Variable types: 2 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 28.0000000
Presolve removed 3 rows and 1 columns
Presolve time: 0.00s
Presolved: 10 rows, 19 columns, 52 nonzeros
Variable types: 0 continuous, 19 integer (18 binary)

Root relaxation: cutoff, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        28.00000   28.00000

Balance Constrain: Balance the means, second moments (variance), and cross-product correlation of two covariates

In [27]:
import gurobipy as gp
from gurobipy import GRB

# Define sets
T = [1, 2, 3]  # Treated units
C = [4, 5, 6, 7, 8, 9]  # Control units
P = [1, 2]  # Covariates (p1 and p2)
m = 2  # Number of controls per treated unit

# Example distances δ_t,c
delta = {(t, c): abs(t - c) for t in T for c in C}

# Example covariate values (randomized)
x = {
    (4, 1): 10, (4, 2): 20, (5, 1): 15, (5, 2): 25,
    (6, 1): 12, (6, 2): 22, (7, 1): 11, (7, 2): 21,
    (8, 1): 14, (8, 2): 24, (9, 1): 13, (9, 2): 23
}

# Compute moments for treated units (Example)
x_T = {1: 13, 2: 23}  # Mean
x2_T = {1: 170, 2: 530}  # Second moment (squared)
x_cross_T = 299  # Mean of cross-product x_p1 * x_p2

# Covariate importance weights (Ensure all keys 1 to 5 exist)
omega = {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}

# Create Gurobi model
model = gp.Model("Optimal_Matching_Multivariate")

# Decision variables
a = model.addVars(T, C, vtype=GRB.BINARY, name="a")
z = model.addVars(range(1, 6), vtype=GRB.CONTINUOUS, name="z")  # Ensured indices 1 to 5

# Objective function: Minimize sum of distances and covariate imbalances
model.setObjective(
    gp.quicksum(delta[t, c] * a[t, c] for t in T for c in C) + gp.quicksum(omega[i] * z[i] for i in range(1, 6)),
    GRB.MINIMIZE
)

# Constraint: Each treated unit is matched to exactly m controls
for t in T:
    model.addConstr(gp.quicksum(a[t, c] for c in C) == m, f"Match_{t}")

# Constraint: Each control is used at most once
for c in C:
    model.addConstr(gp.quicksum(a[t, c] for t in T) <= 1, f"Control_{c}")

# Covariate balance constraints (Means, Second Moments, Cross Product)
model.addConstr(
    z[1] >= gp.quicksum(a[t, c] * x[c, 1] for t in T for c in C) / (m * len(T)) - x_T[1],
    "Mean_Balance_p1_Pos"
)
model.addConstr(
    z[1] >= -gp.quicksum(a[t, c] * x[c, 1] for t in T for c in C) / (m * len(T)) + x_T[1],
    "Mean_Balance_p1_Neg"
)

model.addConstr(
    z[2] >= gp.quicksum(a[t, c] * x[c, 1]**2 for t in T for c in C) / (m * len(T)) - x2_T[1],
    "SecondMoment_Balance_p1_Pos"
)
model.addConstr(
    z[2] >= -gp.quicksum(a[t, c] * x[c, 1]**2 for t in T for c in C) / (m * len(T)) + x2_T[1],
    "SecondMoment_Balance_p1_Neg"
)

model.addConstr(
    z[3] >= gp.quicksum(a[t, c] * x[c, 2] for t in T for c in C) / (m * len(T)) - x_T[2],
    "Mean_Balance_p2_Pos"
)
model.addConstr(
    z[3] >= -gp.quicksum(a[t, c] * x[c, 2] for t in T for c in C) / (m * len(T)) + x_T[2],
    "Mean_Balance_p2_Neg"
)

model.addConstr(
    z[4] >= gp.quicksum(a[t, c] * x[c, 2]**2 for t in T for c in C) / (m * len(T)) - x2_T[2],
    "SecondMoment_Balance_p2_Pos"
)
model.addConstr(
    z[4] >= -gp.quicksum(a[t, c] * x[c, 2]**2 for t in T for c in C) / (m * len(T)) + x2_T[2],
    "SecondMoment_Balance_p2_Neg"
)

model.addConstr(
    z[5] >= gp.quicksum(a[t, c] * x[c, 1] * x[c, 2] for t in T for c in C) / (m * len(T)) - x_cross_T,
    "CrossProduct_Balance_Pos"
)
model.addConstr(
    z[5] >= -gp.quicksum(a[t, c] * x[c, 1] * x[c, 2] for t in T for c in C) / (m * len(T)) + x_cross_T,
    "CrossProduct_Balance_Neg"
)

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal Matching Found:")
    for t in T:
        for c in C:
            if a[t, c].x > 0.5:
                print(f"Treated {t} matched with Control {c}")
    print("\nCovariate imbalance values:")
    for i in range(1, 6):
        print(f"z[{i}] = {z[i].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19 rows, 23 columns and 226 nonzeros
Model fingerprint: 0x09b7484f
Variable types: 5 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 74.5000000
Presolve removed 6 rows and 1 columns
Presolve time: 0.04s
Presolved: 13 rows, 22 columns, 100 nonzeros
Variable types: 0 continuous, 22 integer (18 binary)

Root relaxation: infeasible, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0        74.50000   74.

Balance Constrain: Incorporate quantile balance and the Kolmogorov-Smirnov (K-S) statistic

In [9]:
import gurobipy as gp
from gurobipy import GRB

# Define sets
T = [1, 2, 3]  # Treated units
C = [4, 5, 6, 7, 8, 9]  # Control units
P = [1, 2]  # Covariates
m = 2  # Number of controls per treated unit

# Define quantiles for each covariate
G = {1: [10, 15, 20], 2: [20, 25, 30]}  # Example quantile grid for p1 and p2

# Example distances δ_t,c
delta = {(t, c): abs(t - c) for t in T for c in C}

# Example covariate values (randomized)
x = {
    (4, 1): 10, (4, 2): 20, (5, 1): 15, (5, 2): 25,
    (6, 1): 12, (6, 2): 22, (7, 1): 11, (7, 2): 21,
    (8, 1): 14, (8, 2): 24, (9, 1): 13, (9, 2): 23
}

# Compute empirical CDF for treated units at quantiles h_g
h = {
    1: {10: 0.2, 15: 0.5, 20: 0.8},  # Empirical CDF for covariate 1
    2: {20: 0.25, 25: 0.6, 30: 0.9}  # Empirical CDF for covariate 2
}

# Covariate importance weights
omega = {1: 1.0, 2: 1.0}

# Create Gurobi model
model = gp.Model("Optimal_Matching_KS")

# Decision variables
a = model.addVars(T, C, vtype=GRB.BINARY, name="a")
z = model.addVars(P, vtype=GRB.CONTINUOUS, name="z")

# Objective function: Minimize sum of distances and Kolmogorov-Smirnov imbalances
model.setObjective(
    gp.quicksum(delta[t, c] * a[t, c] for t in T for c in C) +
    gp.quicksum(omega[p] * z[p] for p in P),
    GRB.MINIMIZE
)

# Constraint: Each treated unit is matched to exactly m controls
for t in T:
    model.addConstr(gp.quicksum(a[t, c] for c in C) == m, f"Match_{t}")

# Constraint: Each control is used at most once
for c in C:
    model.addConstr(gp.quicksum(a[t, c] for t in T) <= 1, f"Control_{c}")

# Kolmogorov-Smirnov Constraints
for p in P:
    for g_p in G[p]:
        # Compute indicator function: 1 if x[c, p] < g_p, 0 otherwise
        indicator_sum = gp.quicksum(a[t, c] * (1 if x[c, p] < g_p else 0) for t in T for c in C) / (m * len(T))
        
        # KS Upper Bound Constraint
        model.addConstr(omega[p] * z[p] >= h[p][g_p] - indicator_sum, f"KS_Upper_{p}_{g_p}")

        # KS Lower Bound Constraint
        model.addConstr(omega[p] * z[p] >= -h[p][g_p] + indicator_sum, f"KS_Lower_{p}_{g_p}")

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal Matching Found:")
    for t in T:
        for c in C:
            if a[t, c].x > 0.5:
                print(f"Treated {t} matched with Control {c}")
    print("\nKolmogorov-Smirnov imbalance values:")
    for p in P:
        print(f"z[{p}] = {z[p].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 20 columns and 180 nonzeros
Model fingerprint: 0x750ad862
Variable types: 2 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e+00]
Found heuristic solution: objective 27.5833333
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 20 columns, 84 nonzeros
Variable types: 0 continuous, 20 integer (18 binary)

Root relaxation: cutoff, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        27.58333   27.58333

Balance Constrain: Exact and near-exact matching constraints

In [13]:
import gurobipy as gp
from gurobipy import GRB

# Define sets
T = [1, 2, 3]  # Treated units
C = [4, 5, 6, 7, 8, 9]  # Control units
B = [0, 1]  # Categories of a nominal covariate
m = 2  # Number of controls per treated unit
xi = 2  # Allowed deviation for near-exact matching

# Example categorical covariate values for treated and control groups
x_cat = {
    1: 0, 2: 1, 3: 0,  # Treated
    4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1  # Control
}

# Example distances δ_t,c
delta = {(t, c): abs(t - c) for t in T for c in C}

# Create Gurobi model
model = gp.Model("Optimal_Matching_Exact")

# Decision variables
a = model.addVars(T, C, vtype=GRB.BINARY, name="a")
u = model.addVars(B, vtype=GRB.CONTINUOUS, name="u")  # Auxiliary variables for near-exact matching

# Objective function: Minimize total distance
model.setObjective(
    gp.quicksum(delta[t, c] * a[t, c] for t in T for c in C),
    GRB.MINIMIZE
)

# Constraint: Each treated unit is matched to exactly m controls
for t in T:
    model.addConstr(gp.quicksum(a[t, c] for c in C) == m, f"Match_{t}")

# Constraint: Each control is used at most once
for c in C:
    model.addConstr(gp.quicksum(a[t, c] for t in T) <= 1, f"Control_{c}")

# **Exact Matching Constraint**
for b in B:
    model.addConstr(
        gp.quicksum(a[t, c] * (1 if x_cat[t] == b and x_cat[c] == b else 0) for t in T for c in C)
        == m * sum(1 for t in T if x_cat[t] == b),
        f"Exact_Matching_{b}"
    )

# **Near-Exact Matching Constraint using auxiliary variable**
for b in B:
    model.addConstr(
        u[b] >= gp.quicksum(a[t, c] * (1 if x_cat[t] == b and x_cat[c] == b else 0) for t in T for c in C)
        - m * sum(1 for t in T if x_cat[t] == b),
        f"Near_Exact_Matching_Upper_{b}"
    )

    model.addConstr(
        u[b] >= -gp.quicksum(a[t, c] * (1 if x_cat[t] == b and x_cat[c] == b else 0) for t in T for c in C)
        + m * sum(1 for t in T if x_cat[t] == b),
        f"Near_Exact_Matching_Lower_{b}"
    )

    model.addConstr(u[b] <= xi, f"Near_Exact_Bound_{b}")

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal Matching Found:")
    for t in T:
        for c in C:
            if a[t, c].x > 0.5:
                print(f"Treated {t} matched with Control {c}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 20 columns and 69 nonzeros
Model fingerprint: 0xec2582d3
Variable types: 2 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 8 rows and 6 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


### Balancing Weights

Balancing weight Implementation Using Mixed Integer Programming

In [28]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Simulated dataset
np.random.seed(42)
N = 100  # Number of units
K = 3  # Number of treatment factors

# Generate covariates (X), treatment assignments (Z), and outcomes (Y)
X = np.random.normal(0, 1, (N, 3))  # 3 covariates
Z = np.random.randint(0, 2, (N, K))  # Binary treatment assignments
Y = 2 * Z[:, 0] + 3 * Z[:, 1] - Z[:, 2] + np.random.normal(0, 1, N)  # Outcome model

# Define the balancing constraints
def balance_constraint(w, X, Z):
    """Enforces balance across treatment groups."""
    treated_mean = np.average(X[Z[:, 0] == 1], weights=w[Z[:, 0] == 1], axis=0)
    control_mean = np.average(X[Z[:, 0] == 0], weights=w[Z[:, 0] == 0], axis=0)
    return np.abs(treated_mean - control_mean).sum()

# Define objective function (variance minimization)
def objective(w):
    return np.sum(w**2)  # Minimize variance

# Constraints: balance on covariates and sum of weights
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum to 1
               {'type': 'eq', 'fun': lambda w: balance_constraint(w, X, Z)}]  # Balance constraint

# Bounds (weights must be non-negative)
bounds = [(0, 1)] * N

# Initial weights (uniform)
w0 = np.ones(N) / N

# Solve optimization problem
result = minimize(objective, w0, constraints=constraints, bounds=bounds, method='SLSQP')

# Extract optimal weights
optimal_weights = result.x

# Create DataFrame for visualization
df = pd.DataFrame({'Unit': np.arange(N), 'Weight': optimal_weights, 'Treatment': Z[:, 0], 'Outcome': Y})

# Display results
print(df.head())


   Unit    Weight  Treatment   Outcome
0     0  0.009868          0  1.549935
1     1  0.010261          0 -0.377150
2     2  0.010265          1  0.932380
3     3  0.010393          0  1.857621
4     4  0.008349          1  2.120296


In [30]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import Bounds

# Simulated dataset
np.random.seed(42)
N = 100  # Number of units
K = 3  # Number of treatment factors

# Generate covariates (X), treatment assignments (Z), and outcomes (Y)
X = np.random.normal(0, 1, (N, 3))  # 3 covariates
Z = np.random.randint(0, 2, (N, K))  # Binary treatment assignments
Y = 2 * Z[:, 0] + 3 * Z[:, 1] - Z[:, 2] + np.random.normal(0, 1, N)  # Outcome model

# Define objective function (minimizing weight variance)
def objective(w):
    return np.sum(w**2)  # Minimize variance

# Integer Constraints for Subset Selection
M = 1  # Large constant for binary constraint
z = np.random.randint(0, 2, N)  # Random binary selection (1 for selected, 0 for excluded)

def subset_selection_constraint(w):
    """Constraint ensuring selection of certain units."""
    return M * z - w  # Ensures w_i is zero when z_i = 0

# Define constraints list with only subset selection and sum-to-one constraint
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum to 1 constraint
    {'type': 'ineq', 'fun': subset_selection_constraint}  # Subset selection constraint
]

# Bounds (non-negative weights)
bounds = Bounds([0] * N, [1] * N)

# Initial weights (uniform)
w0 = np.ones(N) / N

# Solve optimization problem
result = minimize(objective, w0, constraints=constraints, bounds=bounds, method='SLSQP')

# Extract optimal weights
optimal_weights = result.x

# Create DataFrame for visualization
df = pd.DataFrame({'Unit': np.arange(N), 'Weight': optimal_weights, 'Treatment': Z[:, 0], 'Outcome': Y})

# Display results
print(df.head())


   Unit    Weight  Treatment   Outcome
0     0  0.018868          0  1.549935
1     1  0.000000          0 -0.377150
2     2  0.018868          1  0.932380
3     3  0.018868          0  1.857621
4     4  0.000000          1  2.120296
